In [1]:
import pandas as pd
import numpy as np
import csv
import os
from lxml import etree
import xml.etree.ElementTree as ET
from Bio import Entrez
from Bio import SeqIO
import time
import datetime

Entrez.email = "mnaffairs.intl@gmail.com"

In [17]:
aaMapThreeToOne = {'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
                   'Glu': 'E', 'Gln': 'Q', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
                   'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
                   'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V'}

In [18]:
def readHumanGenesEC(path):
    genes_dict = {}
    dup = set()
    with open(path, 'r') as f:
        f.readline()
        reader = csv.reader(f)
        for row in reader:
            key = row[0]
            if key in genes_dict.keys():
                dup.add(key)
            else:
                genes_dict[key] = row[1]
    print(f'{len(dup)} genes are duplicated: ', dup)
    return genes_dict

In [19]:
genes_dict = readHumanGenesEC('../data/HumanEnzWithEC.csv')
print(f'{len(genes_dict)} genes are imported')

10 genes are duplicated:  {'ERVK-10', '', 'ERVK-18', 'HERVK_113', 'ERVK-7', 'ERVK-19', 'ERVK-6', 'ERVK-8', 'ERVK-25', 'ERVK-9'}
4346 genes are imported


In [20]:
class variationHandler(object):
    # this class is passed to XMLparser to parse an xml file including info on mutations
    def __init__(self, genes_dict):
        self.dictlist = {'gene_ID':[], 'gene_name':[], 'clinical_significance': [], 'EC_number': [], 'missense_variation': [], 'NC_accession': [], 'NP_accession': [], 'ClinVar_accession':[], 'Chr': [], 'start':[], 'stop':[], 'referenceAllele':[], 'alternateAllele':[], 'strand': []}
        self.genes_dict = genes_dict
        self.unnecessary_types = ('inversion', 'copy number gain', 'tandem duplication', 'microsatellite', 'copy number loss', 'distinct chromosomes', 'fusion', 'complex', 'duplication', 'translocation')
        self.is_type = False
        self.is_type_multi = False  # modified
        self.is_simple_allele = False
        self.gene_ID = ""
        self.gene_name = ""
        self.clinvar_acc = ""
        self.nc_acc = ""
        self.np_acc = ""
        self.change = ""
        self.chr = ""
        self.start_num = ""
        self.stop_num = ""
        self.referenceAllele = ""
        self.alternateAllele = ""
        self.is_GeneList = False
        self.ct_gene = 0
        self.strand_dict = {}  
        self.is_hgvs_grch38 = False
        self.ct_np = 0
        self.ct_mc = 0  # counter for Molecular Consequence tag
        self.is_missense = False
        self.is_conflicting = False
        self.is_not_provided = False
        self.is_no_intpn_add = False
        self.is_interpretations = False
        self.is_interpretation = False
        self.is_description = False
        self.is_desc_hist = False
        self.intpn = []
        self.no_intpn = []
        self.intpn_var = []
        self.ct = 0
        self.ct_missense = 0
        self.ct_uncertain = 0
        self.ct_conflicting = 0
        self.ct_not_provided = 0
        self.ct_no_intpn = 0
        self.ct_no_intpn_added = 0
        self.allele_set = set()
        self.allele_set_inside = set()
        
    def start(self, tag, attrs):
        if (tag == 'VariationArchive') and (attrs.get('VariationType').lower() not in self.unnecessary_types):
            self.is_type = True
            if attrs.get('VariationType').lower() == 'haplotype'  or attrs.get('VariationType').lower() == 'diplotype' or attrs.get('VariationType').lower() == 'compoundheterozygote':
                self.is_type_multi = True
            self.clinvar_acc = attrs.get('Accession')
        if self.is_type:
            if tag == 'SimpleAllele':
                variation_id = attrs.get('VariationID')
                if self.is_type_multi == False:
                    if variation_id not in self.allele_set:
                        self.allele_set.add(variation_id)
                    elif variation_id is not None:
                        print('found a duplicated VariationID: ', variation_id)
                else:
                    if variation_id is not None:
                        self.allele_set_inside.add(variation_id)
            elif tag == 'GeneList':
                self.is_GeneList = True
            elif tag == 'Gene' and self.ct_gene == 0:
                self.gene_ID = attrs.get('Symbol')
                self.gene_name = attrs.get('FullName')
                self.ct_gene += 1
            elif tag == 'SequenceLocation' and self.is_GeneList == True and self.ct_gene == 1:
                if attrs.get('Assembly') == 'GRCh38':
                    nc_accession = attrs.get('Accession')
                    strand = attrs.get('Strand')
                    self.strand_dict[nc_accession] = strand
            elif tag == 'SequenceLocation' and self.is_GeneList == False:
                if attrs.get('Assembly') == 'GRCh38':
                    self.chr = attrs.get('Chr')
                    self.start_num = attrs.get('start')
                    self.stop_num = attrs.get('stop')
                    self.referenceAllele = attrs.get('referenceAlleleVCF')
                    self.alternateAllele = attrs.get('alternateAlleleVCF')
            elif tag == 'HGVS' and attrs.get('Assembly') == 'GRCh38':
                self.is_hgvs_grch38 = True
            elif tag == 'NucleotideExpression' and self.is_hgvs_grch38 == True:
                self.nc_acc = attrs.get('sequenceAccessionVersion')
            elif tag == 'ProteinExpression' and self.ct_np == 0:
                self.np_acc = attrs.get('sequenceAccessionVersion')
                self.change = attrs.get('change') 
                if self.np_acc and self.np_acc.startswith('NP'):            
                    self.ct_np += 1
            elif tag == 'MolecularConsequence' and self.ct_mc == 0:
                if attrs.get('Type') and 'missense' in attrs.get('Type').lower():
                    self.is_missense = True
                    self.ct_missense += 1
                self.ct_mc += 1
            elif tag == 'Interpretations':
                self.is_interpretations = True
            elif tag == 'Interpretation':
                self.is_interpretation = True
            elif tag == 'Description':
                self.is_description = True
            elif tag == 'DescriptionHistory':
                self.is_desc_hist = True
            elif tag == 'InterpretedVariation':
                self.intpn_var.append(attrs['Accession'])
            
    def end(self, tag):
        if (tag == 'VariationArchive' and self.is_type and (not self.is_type_multi)):
            if len(self.intpn) == 1:
                clinical_significance = self.intpn[0].lower()
                if "uncertain" in clinical_significance:
                    self.is_uncertain = True
                    self.ct_uncertain += 1
                elif "conflicting" in clinical_significance:
                    self.is_conflicting = True
                    self.ct_conflicting += 1
                elif "not provided" in clinical_significance:
                    self.is_not_provided = True
                    self.ct_not_provided += 1
                elif clinical_significance == 'no interpretation for the single variant':
                    if set(self.intpn_var).issubset(set(self.dictlist['ClinVar_accession'])):
                        self.is_no_intpn_add = True
                        self.ct_no_intpn_added += 1
                    else:
                        self.no_intpn.append(self.clinvar_acc)  #record all clinvar_acc of varations with no interpretaion to check if there are any variants whose haplotype's interpreation cannot be checked because of the order of the xml file
                    self.ct_no_intpn += 1
            elif len(self.intpn) > 1:
                print(f'Interpretaion: {self.intpn}, Accession: {self.clinvar_acc}, count: {self.ct}')
            if (self.gene_ID in self.genes_dict.keys()) and self.is_missense and (self.is_uncertain or self.is_conflicting or self.is_not_provided or self.is_no_intpn_add):
                try:
                    self.change = self.change.split('p.')[1]
                    before = aatranlation.get(self.change[0:3])
                    after = aatranlation.get(self.change[len(self.change) - 3:len(self.change)])
                except: 
                    before = None
                    after = None
                if before and after:  # check if both have a value in aa dict
                    num = self.change[3:len(self.change) - 3]
                    abbreviated_change = before + num + after
                    fasta = np.nan
                    self.dictlist['gene_ID'].append(self.gene_ID)
                    self.dictlist['gene_name'].append(self.gene_name)
                    self.dictlist['clinical_significance'].append(clinical_significance)
                    self.dictlist['EC_number'].append(self.genes_dict.get(self.gene_ID))
                    self.dictlist['missense_variation'].append(abbreviated_change)
                    self.dictlist['NP_accession'].append(self.np_acc)
                    self.dictlist['NC_accession'].append(self.nc_acc)
                    self.dictlist['ClinVar_accession'].append(self.clinvar_acc)
                    self.dictlist['Chr'].append(self.chr)
                    self.dictlist['start'].append(self.start_num)
                    self.dictlist['stop'].append(self.stop_num)
                    self.dictlist['referenceAllele'].append(self.referenceAllele)
                    self.dictlist['alternateAllele'].append(self.alternateAllele)
                    strand = self.strand_dict.get(self.nc_acc)
                    self.dictlist['strand'].append(strand)
            self.ct_gene = 0             
            self.is_missense = False
            self.is_uncertain = False
            self.is_conflicting = False
            self.is_not_provided = False
            self.is_no_intpn_add = False
            self.ct_np = 0
            self.ct_mc = 0
            self.intpn = []
            self.is_type = False
            self.is_type_multi = False
            self.ct +=1
            if self.ct % 10000 == 0:
                print(f'counter: {self.ct}')
        if self.is_type:
            if tag == 'GeneList':
                self.is_GeneList = False
            elif tag == 'Interpretations':
                self.is_interpretations = False
            elif tag == 'Interpretation':
                self.is_interpretaion = False
            elif tag == 'Description':
                self.is_description = False
            elif tag == 'DescriptionHistory':
                self.is_desc_hist = False
            elif tag == 'HGVS' and self.is_hgvs_grch38 == True:
                self.is_hgvs_grch38 = False
                
    def data(self, data):
        if (not self.is_simple_allele) and self.is_interpretations and self.is_interpretation and self.is_description and (not self.is_desc_hist):
            self.intpn.append(data)
            
    def close(self):
        print(f"Variations: {self.ct}")
        print(f"Uncertain Significance: {self.ct_uncertain}")
        print(f"Conflicting Report: {self.ct_conflicting}")
        print(f"Not Provided: {self.ct_not_provided}")
        print(f"No interpretation for the single variant: {self.ct_no_intpn_added}")
        print(f"Missense: {self.ct_missense}")
        print(f"No interpretations that aren't included: {self.no_intpn}")
        print(f"Mutations in the list: {len(self.dictlist['gene_ID'])}")
        print(f"")
        print('debug: the file is closed')
        return self.dictlist, self.allele_set, self.allele_set_inside

In [21]:
# class variationHandler(object):
#     def __init__(self, genes_dict):
#         self.dictlist = {'gene_ID':[], 'gene_name':[], 'clinical_significance': [], 'EC_number': [], 'missense_variation': [], 'NC_accession': [], 'NP_accession': [], 'ClinVar_accession':[], 'Chr': [], 'start':[], 'stop':[], 'referenceAllele':[], 'alternateAllele':[], 'strand': []}
#         self.genes_dict = genes_dict
#         self.unnecessary_types = ('inversion', 'copy number gain', 'tandem duplication', 'microsatellite', 'copy number loss', 'distinct chromosomes', 'fusion', 'complex', 'duplication', 'translocation')
#         self.is_type = False
#         self.is_simple_allele = False
#         self.gene_ID = ""
#         self.gene_name = ""
#         self.clinvar_acc = ""
#         self.nc_acc = ""
#         self.np_acc = ""
#         self.change = ""
#         self.chr = ""
#         self.start_num = ""
#         self.stop_num = ""
#         self.referenceAllele = ""
#         self.alternateAllele = ""
#         self.is_GeneList = False
#         self.ct_gene = 0
#         self.strand_dict = {}  
#         self.is_hgvs_grch38 = False
#         self.ct_np = 0
#         self.ct_mc = 0  # counter for Molecular Consequence tag
#         self.is_haplotype = False  # check if a variation is haplotype or not
#         self.is_genotype = False  # check if a variation is genotype or not
#         self.is_missense = False
#         self.is_conflicting = False
#         self.is_not_provided = False
#         self.is_no_intpn_add = False
#         self.is_interpretations = False
#         self.is_interpretation = False
#         self.is_description = False
#         self.is_desc_hist = False
#         self.intpn = []
#         self.no_intpn = []
#         self.intpn_var = []
#         self.ct = 0
#         self.ct_missense = 0
#         self.ct_uncertain = 0
#         self.ct_conflicting = 0
#         self.ct_not_provided = 0
#         self.ct_no_intpn = 0
#         self.ct_no_intpn_added = 0
#         self.allele_set = set()
        
#     def start(self, tag, attrs):
#         if (tag == 'VariationArchive') and (attrs.get('VariationType').lower() not in self.unnecessary_types):
#             self.is_type = True
#             if attrs.get('VariationType').lower() == 'haplotype'  or attrs.get('VariationType').lower() == 'diplotype':
#                 self.is_haplotype = True
#             if attrs.get('VariationType').lower() == 'compoundheterozygote':
#                 self.is_genotype = True
#             self.clinvar_acc = attrs.get('Accession')
#         if self.is_type:
#             if tag == 'SimpleAllele':
#                 self.is_simple_allele = True
#                 variation_id = attrs.get('VariationID')
#                 if variation_id not in self.allele_set:
#                     self.allele_set.add(variation_id)
#                 elif variation_id is not None:
#                     print('found a duplicated VariationID: ', variation_id)
#             elif tag == 'GeneList':
#                 self.is_GeneList = True
#             elif tag == 'Gene' and self.ct_gene == 0:
#                 self.gene_ID = attrs.get('Symbol')
#                 self.gene_name = attrs.get('FullName')
#                 self.ct_gene += 1
#             elif tag == 'SequenceLocation' and self.is_GeneList == True and self.ct_gene == 1:
#                 if attrs.get('Assembly') == 'GRCh38':
#                     nc_accession = attrs.get('Accession')
#                     strand = attrs.get('Strand')
#                     self.strand_dict[nc_accession] = strand
#             elif tag == 'SequenceLocation' and self.is_GeneList == False:
#                 if attrs.get('Assembly') == 'GRCh38':
#                     self.chr = attrs.get('Chr')
#                     self.start_num = attrs.get('start')
#                     self.stop_num = attrs.get('stop')
#                     self.referenceAllele = attrs.get('referenceAlleleVCF')
#                     self.alternateAllele = attrs.get('alternateAlleleVCF')
#             elif tag == 'HGVS' and attrs.get('Assembly') == 'GRCh38':
#                 self.is_hgvs_grch38 = True
#             elif tag == 'NucleotideExpression' and self.is_hgvs_grch38 == True:
#                 self.nc_acc = attrs.get('sequenceAccessionVersion')
#             elif tag == 'ProteinExpression' and self.ct_np == 0:
#                 self.np_acc = attrs.get('sequenceAccessionVersion')
#                 self.change = attrs.get('change') 
#                 if self.np_acc and self.np_acc.startswith('NP'):            
#                     self.ct_np += 1
#             elif tag == 'MolecularConsequence' and self.ct_mc == 0:
#                 if attrs.get('Type') and 'missense' in attrs.get('Type').lower():
#                     self.is_missense = True
#                     self.ct_missense += 1
#                 self.ct_mc += 1
#             elif tag == 'Interpretations':
#                 self.is_interpretations = True
#             elif tag == 'Interpretation':
#                 self.is_interpretation = True
#             elif tag == 'Description':
#                 self.is_description = True
#             elif tag == 'DescriptionHistory':
#                 self.is_desc_hist = True
#             elif tag == 'InterpretedVariation':
#                 self.intpn_var.append(attrs['Accession'])
            
#     def end(self, tag):
#         if (tag == 'VariationArchive' and self.is_type) or ((self.is_haplotype or self.is_genotype) and tag == 'SimpleAllele'):
#             if len(self.intpn) == 1:
#                 clinical_significance = self.intpn[0].lower()
#                 if "uncertain" in clinical_significance:
#                     self.is_uncertain = True
#                     self.ct_uncertain += 1
#                 elif "conflicting" in clinical_significance:
#                     self.is_conflicting = True
#                     self.ct_conflicting += 1
#                 elif "not provided" in clinical_significance:
#                     self.is_not_provided = True
#                     self.ct_not_provided += 1
#                 elif clinical_significance == 'no interpretation for the single variant':
#                     if set(self.intpn_var).issubset(set(self.dictlist['ClinVar_accession'])):
#                         self.is_no_intpn_add = True
#                         self.ct_no_intpn_added += 1
#                     else:
#                         self.no_intpn.append(self.clinvar_acc)  #record all clinvar_acc of varations with no interpretaion to check if there are any variants whose haplotype's interpreation cannot be checked because of the order of the xml file
#                     self.ct_no_intpn += 1
#             elif len(self.intpn) > 1:
#                 print(f'Interpretaion: {self.intpn}, Accession: {self.clinvar_acc}, count: {self.ct}')
#             if (self.gene_ID in self.genes_dict.keys()) and self.is_missense and (self.is_uncertain or self.is_conflicting or self.is_not_provided or self.is_no_intpn_add):
#                 try:
#                     self.change = self.change.split('p.')[1]
#                     before = aatranlation.get(self.change[0:3])
#                     after = aatranlation.get(self.change[len(self.change) - 3:len(self.change)])
#                 except: 
#                     before = None
#                     after = None
#                 if before and after:  # check if both have a value in aa dict
#                     num = self.change[3:len(self.change) - 3]
#                     abbreviated_change = before + num + after
#                     fasta = np.nan
#                     self.dictlist['gene_ID'].append(self.gene_ID)
#                     self.dictlist['gene_name'].append(self.gene_name)
#                     self.dictlist['clinical_significance'].append(clinical_significance)
#                     self.dictlist['EC_number'].append(self.genes_dict.get(self.gene_ID))
#                     self.dictlist['missense_variation'].append(abbreviated_change)
#                     self.dictlist['NP_accession'].append(self.np_acc)
#                     self.dictlist['NC_accession'].append(self.nc_acc)
#                     self.dictlist['ClinVar_accession'].append(self.clinvar_acc)
#                     self.dictlist['Chr'].append(self.chr)
#                     self.dictlist['start'].append(self.start_num)
#                     self.dictlist['stop'].append(self.stop_num)
#                     self.dictlist['referenceAllele'].append(self.referenceAllele)
#                     self.dictlist['alternateAllele'].append(self.alternateAllele)
#                     strand = self.strand_dict.get(self.nc_acc)
#                     self.dictlist['strand'].append(strand)
#             self.ct_gene = 0             
#             self.is_missense = False
#             self.is_uncertain = False
#             self.is_conflicting = False
#             self.is_not_provided = False
#             self.is_no_intpn_add = False
#             self.ct_np = 0
#             self.ct_mc = 0
#             self.intpn = []
#         if self.is_type:
#             if tag == 'SimpleAllele':
#                 self.is_simple_allele = False
#             elif tag == 'GeneList':
#                 self.is_GeneList = False
#             elif tag == 'Interpretations':
#                 self.is_interpretations = False
#             elif tag == 'Interpretation':
#                 self.is_interpretaion = False
#             elif tag == 'Description':
#                 self.is_description = False
#             elif tag == 'DescriptionHistory':
#                 self.is_desc_hist = False
#             elif tag == 'HGVS' and self.is_hgvs_grch38 == True:
#                 self.is_hgvs_grch38 = False
#         if tag == 'VariationArchive':
#             self.is_type = False
#             self.is_haplotype = False
#             self.is_genotype = False
#             self.ct +=1
#             if self.ct % 10000 == 0:
#                 print(f'counter: {self.ct}')
                
#     def data(self, data):
#         if (not self.is_simple_allele) and self.is_interpretations and self.is_interpretation and self.is_description and (not self.is_desc_hist):
#             self.intpn.append(data)
            
#     def close(self):
#         print(f"Variations: {self.ct}")
#         print(f"Uncertain Significance: {self.ct_uncertain}")
#         print(f"Conflicting Report: {self.ct_conflicting}")
#         print(f"Not Provided: {self.ct_not_provided}")
#         print(f"No interpretation for the single variant: {self.ct_no_intpn_added}")
#         print(f"Missense: {self.ct_missense}")
#         print(f"No interpretations that aren't included: {self.no_intpn}")
#         print(f"Mutations in the list: {len(self.dictlist['gene_ID'])}")
#         print('debug: the file is closed')
#         return self.dictlist

In [22]:
# # read xml file of variations from ClinVar
# # return dataframe and write to a csv file
# def readClinVarVariationsXML(input_path, output_path, gene_set):
#     print('debug: start parcing')
#     parser = etree.XMLParser(target=variationHandler(gene_set))
#     data = etree.parse(input_path, parser)
#     df = pd.DataFrame(data)
#     df.to_csv(output_path, index = False, header = True)
#     return df

In [23]:
# read xml file of variations from ClinVar
# return dataframe and write to a csv file
def readClinVarVariationsXML(input_path, output_path, gene_set):
    print('debug: start parcing')
    parser = etree.XMLParser(target=variationHandler(gene_set))
    data, allele_set, allele_set_inside = etree.parse(input_path, parser)
    df = pd.DataFrame(data)
    df.to_csv(output_path, index = False, header = True)
    return df, allele_set, allele_set_inside

In [11]:
## parse a part of the xml file
xmlfile = '../data/clinvartail_5.xml'
out_path = '../data/MM_enzyme_short.csv'
df_short, allele_set, allele_set_inside = readClinVarVariationsXML(xmlfile, out_path, genes_dict)
print(len(allele_set), len(allele_set_inside))
print(allele_set_inside - allele_set)
df_short.head()

debug: start parcing
Variations: 355
Uncertain Significance: 190
Conflicting Report: 0
Not Provided: 0
No interpretation for the single variant: 0
Missense: 204
No interpretations that aren't included: []
Mutations in the list: 16

debug: the file is closed
356 125
{'242771', '830294', '830244', '830272', '830248', '830301', '830315', '830350', '830259', '830252', '830303', '830335', '830260', '830306', '830300', '830280', '830239', '830342', '830311', '830363', '830240', '830341', '830351', '830245', '830289', '830284', '830302', '830353', '830362', '830288', '830331', '830276', '830254', '830281', '830359', '830354', '830269', '830282', '830366', '830317', '39379', '830278', '830242', '830286', '830295', '830367', '830290', '830298', '830285', '830268', '830352', '830304', '830287', '830348', '830296', '830291', '830270', '830305', '830333', '830297', '830261', '830277', '830299', '830236', '830283', '830334', '830337', '830271', '830339', '830365', '830275', '830265', '830371', '830

,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NC_accession,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,strand
0,CHD2,chromodomain helicase DNA binding protein 2,uncertain significance,3.6.4.12,R178G,NC_000015.10,NP_001036037.1,VCV000829865,15,92937606,92937606,A,G,+
1,SPATA5,spermatogenesis associated 5,uncertain significance,3.6.4.10,P813S,NC_000004.12,NP_001332785.1,VCV000829872,4,123256115,123256115,C,T,+
2,CFB,complement factor B,uncertain significance,3.4.21.47,R143C,NC_000006.12,NP_001701.2,VCV000829882,6,31947135,31947135,C,T,+
3,EYA1,EYA transcriptional coactivator and phosphatase 1,uncertain significance,3.1.3.16; 3.1.3.48,D330Y,NC_000008.11,NP_000494.2,VCV000829887,8,71269802,71269802,C,A,-
4,CFI,complement factor I,uncertain significance,3.4.21.45,C284G,NC_000004.12,NP_000195.3,VCV000829894,4,109760303,109760303,A,C,-


In [103]:
## parse the entire xml file
xmlfile = '../data/ClinVarVariationRelease_00-latest_weekly.xml'
out_path = '../data/MM_enzyme.csv'
df_0, allele_set, allele_set_inside  = readClinVarVariationsXML(xmlfile, out_path, genes_dict)
print(len(allele_set), len(allele_set_inside))
print(allele_set_inside - allele_set)

debug: start parcing
Variations: 2458
Uncertain Significance: 65
Conflicting Report: 3
Not Provided: 17
No interpretation for the single variant: 0
Missense: 407
No interpretations that aren't included: []
Mutations in the list: 12

debug: the file is closed


KeyboardInterrupt: 

In [55]:
ls = ['VCV000012173', 'VCV000025511', 'VCV000132775', 'VCV000132776', 'VCV000242384', 'VCV000242385', 'VCV000242419', 'VCV000242420', 'VCV000242421', 'VCV000242427', 'VCV000242431', 'VCV000242432', 'VCV000242433', 'VCV000242435', 'VCV000242436', 'VCV000242439', 'VCV000242440', 'VCV000242474', 'VCV000242475', 'VCV000242476', 'VCV000242486', 'VCV000242487', 'VCV000242492', 'VCV000242504', 'VCV000242512', 'VCV000242514', 'VCV000242522', 'VCV000242525', 'VCV000242526', 'VCV000242533', 'VCV000242534', 'VCV000242571', 'VCV000242574', 'VCV000242575', 'VCV000242590', 'VCV000242591', 'VCV000242618', 'VCV000242619', 'VCV000242636', 'VCV000242637', 'VCV000242638', 'VCV000242639', 'VCV000242652', 'VCV000242653', 'VCV000242654', 'VCV000242667', 'VCV000242668', 'VCV000242669', 'VCV000242670', 'VCV000242673', 'VCV000242674', 'VCV000242688', 'VCV000242705', 'VCV000242709', 'VCV000242714', 'VCV000242717', 'VCV000242718', 'VCV000242720', 'VCV000242723', 'VCV000242726', 'VCV000242730', 'VCV000242732', 'VCV000242738', 'VCV000242760', 'VCV000242761', 'VCV000242776', 'VCV000242782', 'VCV000242785', 'VCV000242789', 'VCV000242792', 'VCV000242794', 'VCV000242798', 'VCV000242799', 'VCV000242801', 'VCV000242809', 'VCV000242813', 'VCV000242820', 'VCV000264658', 'VCV000264659', 'VCV000264660', 'VCV000264668', 'VCV000264669', 'VCV000264672', 'VCV000264675', 'VCV000267283', 'VCV000267284', 'VCV000267287', 'VCV000267288', 'VCV000267289', 'VCV000267742', 'VCV000267743', 'VCV000279614', 'VCV000279615', 'VCV000374405', 'VCV000374406', 'VCV000375324', 'VCV000375325', 'VCV000375598', 'VCV000375599', 'VCV000375758', 'VCV000375759', 'VCV000393290', 'VCV000393291', 'VCV000393293', 'VCV000393294', 'VCV000393295', 'VCV000393296', 'VCV000417674', 'VCV000417675', 'VCV000424698', 'VCV000428591', 'VCV000428592', 'VCV000429193', 'VCV000429194', 'VCV000430661', 'VCV000431059', 'VCV000431060', 'VCV000440777', 'VCV000441235', 'VCV000446382', 'VCV000446480', 'VCV000478852', 'VCV000478887', 'VCV000478888', 'VCV000478913', 'VCV000478918', 'VCV000478919', 'VCV000487345', 'VCV000487347', 'VCV000545635', 'VCV000549834', 'VCV000560198', 'VCV000560199', 'VCV000560201', 'VCV000617633', 'VCV000617662', 'VCV000617663', 'VCV000624596', 'VCV000624597', 'VCV000624598', 'VCV000624607', 'VCV000624608', 'VCV000624613', 'VCV000635091', 'VCV000635093', 'VCV000635827', 'VCV000635828', 'VCV000689496', 'VCV000242681', 'VCV000624611', 'VCV000624612', 'VCV000427808', 'VCV000427809', 'VCV000478832', 'VCV000478838', 'VCV000478850', 'VCV000478851', 'VCV000242503', 'VCV000370036', 'VCV000242498', 'VCV000242516', 'VCV000242620', 'VCV000242700', 'VCV000242734', 'VCV000242770', 'VCV000242816', 'VCV000242751', 'VCV000446195', 'VCV000496587', 'VCV000242562', 'VCV000242563', 'VCV000242564', 'VCV000242593', 'VCV000242594', 'VCV000242596', 'VCV000242597', 'VCV000635094', 'VCV000635095', 'VCV000635096', 'VCV000635097', 'VCV000560533', 'VCV000585242', 'VCV000242472', 'VCV000242704', 'VCV000242708', 'VCV000242737', 'VCV000242746', 'VCV000242754', 'VCV000242775', 'VCV000242777', 'VCV000242807', 'VCV000242819', 'VCV000242586', 'VCV000242588', 'VCV000242750', 'VCV000242797', 'VCV000446183', 'VCV000446188', 'VCV000692326', 'VCV000692327', 'VCV000242396', 'VCV000242500', 'VCV000242501', 'VCV000242582', 'VCV000242583', 'VCV000242394', 'VCV000242395', 'VCV000242404', 'VCV000242406', 'VCV000242407', 'VCV000242426', 'VCV000242428', 'VCV000242598', 'VCV000242599', 'VCV000242624', 'VCV000242690', 'VCV000242489', 'VCV000242755', 'VCV000242756', 'VCV000242821', 'VCV000267280', 'VCV000267452', 'VCV000267453', 'VCV000267454', 'VCV000267455', 'VCV000267456', 'VCV000267457', 'VCV000267458', 'VCV000267459', 'VCV000267460', 'VCV000267461', 'VCV000267462', 'VCV000267463', 'VCV000375319', 'VCV000375320', 'VCV000478828', 'VCV000478845', 'VCV000478899', 'VCV000518442', 'VCV000561162', 'VCV000624606', 'VCV000375326', 'VCV000375327', 'VCV000631475', 'VCV000631476', 'VCV000039358', 'VCV000242399', 'VCV000242401', 'VCV000242402', 'VCV000242403', 'VCV000242446', 'VCV000242447', 'VCV000242496', 'VCV000242497', 'VCV000242505', 'VCV000242506', 'VCV000242510', 'VCV000242530', 'VCV000242531', 'VCV000242572', 'VCV000242573', 'VCV000242576', 'VCV000242579', 'VCV000242610', 'VCV000242611', 'VCV000242616', 'VCV000242617', 'VCV000242622', 'VCV000242623', 'VCV000242655', 'VCV000242657', 'VCV000242658', 'VCV000242659', 'VCV000242660', 'VCV000242661', 'VCV000242703', 'VCV000242741', 'VCV000242742', 'VCV000242743', 'VCV000242774', 'VCV000267744', 'VCV000267745', 'VCV000375728', 'VCV000375730', 'VCV000417816', 'VCV000438270', 'VCV000438271', 'VCV000438272', 'VCV000487089', 'VCV000488063', 'VCV000488425', 'VCV000488426', 'VCV000488427', 'VCV000627560', 'VCV000627561', 'VCV000635090', 'VCV000635092', 'VCV000684769', 'VCV000684770', 'VCV000242666', 'VCV000560532', 'VCV000021159', 'VCV000050891', 'VCV000050892', 'VCV000050893', 'VCV000050894', 'VCV000050895', 'VCV000050896', 'VCV000242390', 'VCV000242408', 'VCV000242438', 'VCV000242459', 'VCV000242462', 'VCV000242465', 'VCV000242467', 'VCV000242468', 'VCV000242494', 'VCV000242540', 'VCV000242544', 'VCV000242546', 'VCV000242547', 'VCV000242549', 'VCV000242551', 'VCV000242552', 'VCV000242553', 'VCV000242554', 'VCV000242555', 'VCV000242556', 'VCV000242560', 'VCV000242612', 'VCV000242613', 'VCV000242650', 'VCV000242651', 'VCV000242719', 'VCV000242724', 'VCV000242729', 'VCV000242795', 'VCV000242382', 'VCV000242443', 'VCV000242444', 'VCV000242445', 'VCV000242509', 'VCV000242565', 'VCV000242566', 'VCV000242567', 'VCV000242568', 'VCV000242569', 'VCV000242570', 'VCV000242684', 'VCV000242686', 'VCV000242687', 'VCV000242749', 'VCV000242796', 'VCV000264664', 'VCV000264674', 'VCV000267281', 'VCV000267282', 'VCV000402119', 'VCV000430836', 'VCV000430847', 'VCV000430848', 'VCV000429025', 'VCV000429026', 'VCV000430872', 'VCV000430873', 'VCV000433543', 'VCV000433545', 'VCV000433546', 'VCV000433556', 'VCV000433557', 'VCV000446487', 'VCV000444001', 'VCV000444002', 'VCV000446191', 'VCV000545510', 'VCV000545623', 'VCV000545624', 'VCV000559423', 'VCV000559434', 'VCV000624609', 'VCV000624610', 'VCV000624615', 'VCV000636258', 'VCV000636259', 'VCV000637951', 'VCV000691527', 'VCV000691528', 'VCV000694741', 'VCV000694742', 'VCV000242645', 'VCV000242646', 'VCV000242513', 'VCV000242627', 'VCV000242628', 'VCV000431454', 'VCV000560531', 'VCV000585008', 'VCV000691963', 'VCV000242470', 'VCV000242471', 'VCV000242478', 'VCV000242604', 'VCV000242728', 'VCV000242769', 'VCV000242803', 'VCV000264663', 'VCV000264666', 'VCV000264667', 'VCV000268096', 'VCV000268097', 'VCV000393289', 'VCV000393292', 'VCV000397632', 'VCV000478870', 'VCV000478872', 'VCV000478884', 'VCV000478890', 'VCV000478904', 'VCV000478905', 'VCV000487346', 'VCV000559420', 'VCV000559421', 'VCV000590257', 'VCV000590258', 'VCV000624599', 'VCV000624600', 'VCV000624601', 'VCV000624602', 'VCV000624603', 'VCV000624604', 'VCV000624605', 'VCV000684385', 'VCV000684386', 'VCV000813875', 'VCV000813876', 'VCV000242469', 'VCV000242519', 'VCV000007169', 'VCV000208553', 'VCV000208554', 'VCV000242450', 'VCV000242485', 'VCV000242495', 'VCV000242671', 'VCV000242791', 'VCV000264670', 'VCV000264671', 'VCV000267740', 'VCV000267741', 'VCV000373892', 'VCV000375250', 'VCV000375732', 'VCV000430722', 'VCV000437930', 'VCV000440459', 'VCV000446189', 'VCV000478878', 'VCV000559943', 'VCV000559944', 'VCV000818230', 'VCV000818231', 'VCV000242721', 'VCV000478834', 'VCV000487077', 'VCV000487078', 'VCV000487079', 'VCV000487080', 'VCV000487081', 'VCV000827582', 'VCV000827583', 'VCV000242762', 'VCV000242693', 'VCV000242641', 'VCV000242521', 'VCV000242381', 'VCV000242380', 'VCV000375321', 'VCV000375323', 'VCV000446182', 'VCV000634859', 'VCV000634861', 'VCV000634864', 'VCV000634865', 'VCV000634875', 'VCV000634876', 'VCV000634877', 'VCV000634879', 'VCV000634882', 'VCV000830334', 'VCV000830335', 'VCV000830336', 'VCV000830337', 'VCV000830338', 'VCV000830339', 'VCV000830340', 'VCV000830341', 'VCV000830342', 'VCV000830343', 'VCV000830344', 'VCV000830345', 'VCV000830346', 'VCV000830347', 'VCV000830348', 'VCV000830349', 'VCV000830350', 'VCV000830351', 'VCV000830352', 'VCV000830353', 'VCV000830354', 'VCV000024962', 'VCV000024987', 'VCV000050883', 'VCV000050885', 'VCV000132156', 'VCV000242379', 'VCV000242388', 'VCV000242389', 'VCV000242412', 'VCV000242413', 'VCV000242416', 'VCV000242452', 'VCV000242460', 'VCV000242480', 'VCV000242481', 'VCV000242491', 'VCV000242493', 'VCV000242511', 'VCV000242523', 'VCV000242528', 'VCV000242581', 'VCV000242609', 'VCV000242614', 'VCV000242676', 'VCV000242677', 'VCV000242678', 'VCV000242715', 'VCV000242752', 'VCV000242788', 'VCV000242802', 'VCV000242814', 'VCV000264661', 'VCV000267276', 'VCV000375322', 'VCV000375729', 'VCV000375757', 'VCV000424703', 'VCV000425600', 'VCV000425651', 'VCV000440737', 'VCV000440739', 'VCV000446185', 'VCV000446197', 'VCV000478854', 'VCV000487488', 'VCV000487489', 'VCV000487490', 'VCV000487491', 'VCV000493610', 'VCV000549832', 'VCV000549833', 'VCV000549835', 'VCV000549836', 'VCV000549837', 'VCV000549838', 'VCV000618966', 'VCV000626312', 'VCV000691964', 'VCV000830372']
print(len(ls))

568


In [38]:
# # this column is a test for a certain variation I want to look over
# subnode = '../data/subnode.xml'
# temp_path = '../data/temp.csv'
# df_temp = readClinVarVariationsXML(subnode, temp_path, genes_dict)
# df_temp.head()

debug: start parcing
Interpretaion: ['drug response3', 'drug response4'], Accession: VCV000633857, count: 0
Variations: 1
Uncertain Significance: 0
Conflicting Report: 0
Not Provided: 0
No interpretation for the single variant: 0
Missense: 0
No interpretations that aren't included: []
Mutations in the list: 0
debug: the file is closed


,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NC_accession,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,strand


In [19]:
# creates an subnode xml file when you want to look over a certain mutation
class variationHandlerSpecific(object):
    def __init__(self, accession):
        self.is_accession = False
        self.accession = accession
        self.ct = 0
        print(self.accession)
        
    def start(self, tag, attrs):
        global WFILE
        if (tag == 'VariationArchive') and (attrs.get('Accession') == self.accession):
            self.is_accession = True
            print('The specific variation is found: ' + str(self.ct))
        if self.is_accession:
            if len(attrs.keys()) == 0:
                WFILE.write('<' + tag)
            else:   
                for i, t in enumerate(attrs.keys()):
                    if i == 0:
                        WFILE.write('<' + tag + ' ')
                    elif i != len(attrs.keys()) - 1:
                        WFILE.write(t + '="' + attrs.get(t) + '"' + ' ')
                    else:
                        WFILE.write(t + '="' + attrs.get(t) + '"')
            WFILE.write('>')
            
    def end(self, tag):
        global WFILE
        if self.is_accession:
            WFILE.write('</' + tag + '>')
        if tag == 'VariationArchive':
            if self.ct % 10000 == 0:
                print(self.ct)
            self.ct += 1
        if self.is_accession and tag == 'VariationArchive':
            self.is_accession = False
            WFILE.close()
            print('The subnode file is completed')
            
    def data(self, data):
        global WFILE
        if data is not None:
            if self.is_accession and data != "":
                WFILE.write(data)
            
    def close(self):
        print('The xml file is closed')

In [23]:
# read xml file of variations from ClinVar
# return dataframe and creates a subnode xml file
def readClinVarVariationsXMLSpecific(input_path, accession):
    print('Start parcing')
    parser = etree.XMLParser(target=variationHandlerSpecific(accession))
    etree.parse(input_path, parser)

In [20]:
# # this column is to get a certain part of the xml file
# WFILE = open('../data/subnode.xml', 'w')
# xmlfile = '../data/ClinVarVariationRelease_00-latest_weekly.xml'
# accession = 'VCV000424757'
# readClinVarVariationsXMLSpecific(xmlfile, accession)

In [12]:
# makes dictionary of fasta sequences and np number 
# returns the dictionary
def makeDictOfFasta(dictpath):
    fasta_dict = {}
    for root, d_names, file_names in os.walk(dictpath):
        for filename in file_names:
            fname = os.path.join(root, filename)
            with open(fname, 'r') as f:
                print('opened a fasta file')
                np_num = ''
                sequence = ''
                for line in f:
                    if line[0] == '>':
                        if sequence != '':
                            fasta_dict[np_num] = sequence
                            np_num = ''
                            sequence = ''                    
                        i = 1
                        while line[i] != ' ':
                            np_num += line[i]
                            i += 1
                    else:
                        line = line.strip('\n')
                        sequence += line
    print(f'length of fasta dictionary: {len(fasta_dict)}')
    return fasta_dict

In [13]:
fasta_dict = makeDictOfFasta('../fasta_sequences/')

opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
length of fasta dictionary: 113645


In [14]:
# crops fasta sequence
# returns the cropped sequnece with a specified range
def cropFASTA(sequence, location, reference, seqRange):
    if location - 1 < len(sequence) and sequence[location - 1] == reference:
        if location - 1 - seqRange <= 0:
            proteinSeq = sequence[0:2 * seqRange + 1]
        elif location - 1 + seqRange > len(sequence) - 1:
            proteinSeq = sequence[0 if len(sequence) - 1 - 2 * seqRange <= 0 else len(sequence) - 1 - 2 * seqRange:len(sequence)]
        else:
            proteinSeq = sequence[location - 1 - seqRange : location + seqRange]
        return proteinSeq
    else:
        return None 

In [15]:
def addFASTAfromDict(fasta_dict, df):
    unfound = set()
    seq_list = []
    for index, row in df.iterrows():
        mutation = row['missense_variation']
        try:
            ref = mutation[0]
            try:
                location = int(mutation[1:len(mutation)-1])
            except:
                location = int(mutation.split('_')[0][1:])
            np_num = row['NP_accession']  # specify the column of np 
            sequence = fasta_dict[np_num]  # 
            seqRange = 12  # range of sequences to take
            seq = cropFASTA(sequence, location, ref, seqRange) if sequence else None
        except:
            seq = None
            unfound_np = row['NP_accession']
            unfound.add(unfound_np)
        seq_list.append(seq)
    df['FASTA_window'] = seq_list
    print(f'Unfound Sequences: {len(unfound)} {unfound}')
    return df, unfound

In [25]:
df_0 = pd.read_csv('../data/MM_enzyme.csv')
df_0.head()

,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NC_accession,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,strand
0,AGA,aspartylglucosaminidase,uncertain significance,3.5.1.26,G60D,NC_000004.12,NP_000018.2,VCV000000222,4,177440375.0,177440375.0,C,T,-
1,DPYD,dihydropyrimidine dehydrogenase,uncertain significance,1.3.1.2,R886H,NC_000001.11,NP_000101.2,VCV000000437,1,97098598.0,97098598.0,C,T,+
2,PTS,6-pyruvoyltetrahydropterin synthase,uncertain significance,4.2.3.12,R16C,NC_000011.10,NP_000308.1,VCV000000477,11,112226489.0,112226489.0,C,T,+
3,PROC,"protein C, inactivator of coagulation factors ...",uncertain significance,3.4.21.69,P210L,NC_000002.12,NP_000303.1,VCV000000661,2,127426178.0,127426178.0,C,T,+
4,ARSB,arylsulfatase B,uncertain significance,3.1.6.12,G137V,NC_000005.10,NP_000037.2,VCV000000877,5,78969095.0,78969095.0,C,A,-


In [26]:
df_1 = addFASTAfromDict(fasta_dict, df_0)
df_1.to_csv('../data/MM_enzyme.csv', index = False, header = True)
df_1.head()

Unfound Sequences: 23 {'NP_001138503.1', 'NP_000114.2', 'NP_001273003.1', 'NP_004251.3', 'NP_001180240.1', 'NP_000439.1', 'NP_001177317.1', 'NP_001202.4', 'NP_001017975.4', 'NP_001276896.1', 'NP_001041636.1', 'NP_065829.3', 'LRG_319p1', 'NP_000299.2', 'LRG_789p1', 'NP_945347.2', 'NP_001191354.1', 'NP_000342.2', 'NP_001017973.1', 'NP_004597.2', 'NP_000274.2', 'NP_000195.2', 'NP_000450.2'}


,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NC_accession,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,strand,FASTA_window
0,AGA,aspartylglucosaminidase,uncertain significance,3.5.1.26,G60D,NC_000004.12,NP_000018.2,VCV000000222,4,177440375.0,177440375.0,C,T,-,ASGGSALDAVESGCAMCEREQCDGS
1,DPYD,dihydropyrimidine dehydrogenase,uncertain significance,1.3.1.2,R886H,NC_000001.11,NP_000101.2,VCV000000437,1,97098598.0,97098598.0,C,T,+,KKLPSFGPYLEQRKKIIAENKIRLK
2,PTS,6-pyruvoyltetrahydropterin synthase,uncertain significance,4.2.3.12,R16C,NC_000011.10,NP_000308.1,VCV000000477,11,112226489.0,112226489.0,C,T,+,EGGGRRCQAQVSRRISFSASHRLYS
3,PROC,"protein C, inactivator of coagulation factors ...",uncertain significance,3.4.21.69,P210L,NC_000002.12,NP_000303.1,VCV000000661,2,127426178.0,127426178.0,C,T,+,KRDTEDQEDQVDPRLIDGKMTRRGD
4,ARSB,arylsulfatase B,uncertain significance,3.1.6.12,G137V,NC_000005.10,NP_000037.2,VCV000000877,5,78969095.0,78969095.0,C,A,-,DEKLLPQLLKEAGYTTHMVGKWHLG


In [27]:
# some of the fasta sequences are not included in the fasta files so get them online
def getFASTA(ls_np):
    fasta_dict = {}
    for np_num in ls_np:
        handle = Entrez.efetch(db='protein', id=np_num, rettype='fasta', retmode='text', api_key='2959e9bc88ce27224b70cada27e5b58c6b09')
        seq_record = SeqIO.read(handle, 'fasta')
        sequence = seq_record.seq
        fasta_dict[np_num] = sequence
    return fasta_dict

In [28]:
unfound_nps = ('NP_000114.2', 'NP_001017973.1', 'NP_000274.2','NP_000299.2', 'NP_001138503.1', 'NP_004251.3', 'NP_001276896.1', 'NP_001180240.1', 'NP_000195.2', 'NP_001017975.4', 'NP_001202.4', 'NP_945347.2', 'NP_001273003.1', 'NP_000439.1', 'NP_000450.2', 'NP_065829.3', 'NP_004597.2', 'NP_001191354.1', 'NP_001041636.1', 'NP_000342.2', 'NP_001177317.1')
add_fasta = getFASTA(unfound_nps)
fasta_dict.update(add_fasta)

In [29]:
df_1 = addFASTAfromDict(fasta_dict, df_0)
df_1.to_csv('../data/MM_enzyme.csv', index = False, header = True)
df_1.head()

Unfound Sequences: 2 {'LRG_319p1', 'LRG_789p1'}


,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NC_accession,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,strand,FASTA_window
0,AGA,aspartylglucosaminidase,uncertain significance,3.5.1.26,G60D,NC_000004.12,NP_000018.2,VCV000000222,4,177440375.0,177440375.0,C,T,-,ASGGSALDAVESGCAMCEREQCDGS
1,DPYD,dihydropyrimidine dehydrogenase,uncertain significance,1.3.1.2,R886H,NC_000001.11,NP_000101.2,VCV000000437,1,97098598.0,97098598.0,C,T,+,KKLPSFGPYLEQRKKIIAENKIRLK
2,PTS,6-pyruvoyltetrahydropterin synthase,uncertain significance,4.2.3.12,R16C,NC_000011.10,NP_000308.1,VCV000000477,11,112226489.0,112226489.0,C,T,+,EGGGRRCQAQVSRRISFSASHRLYS
3,PROC,"protein C, inactivator of coagulation factors ...",uncertain significance,3.4.21.69,P210L,NC_000002.12,NP_000303.1,VCV000000661,2,127426178.0,127426178.0,C,T,+,KRDTEDQEDQVDPRLIDGKMTRRGD
4,ARSB,arylsulfatase B,uncertain significance,3.1.6.12,G137V,NC_000005.10,NP_000037.2,VCV000000877,5,78969095.0,78969095.0,C,A,-,DEKLLPQLLKEAGYTTHMVGKWHLG


In [30]:
df_1[['NP_accession', 'gene_ID', 'gene_name', 'FASTA_window']].head()

,NP_accession,gene_ID,gene_name,FASTA_window
0,NP_000018.2,AGA,aspartylglucosaminidase,ASGGSALDAVESGCAMCEREQCDGS
1,NP_000101.2,DPYD,dihydropyrimidine dehydrogenase,KKLPSFGPYLEQRKKIIAENKIRLK
2,NP_000308.1,PTS,6-pyruvoyltetrahydropterin synthase,EGGGRRCQAQVSRRISFSASHRLYS
3,NP_000303.1,PROC,"protein C, inactivator of coagulation factors ...",KRDTEDQEDQVDPRLIDGKMTRRGD
4,NP_000037.2,ARSB,arylsulfatase B,DEKLLPQLLKEAGYTTHMVGKWHLG


In [31]:
# make FASTA format text file from dataframe for blast search
def makeFASTAfile(df, output_path):
    subset = df[['NP_accession', 'gene_ID', 'gene_name', 'FASTA_window']]
    tuples = [tuple(x) for x in subset.values]
    with open(output_path, 'w') as f:
        for tup in tuples:
            line = '>' + '\t'.join(tup[0:3]) + '\n'
            fasta = str(tup[3]) + '\n'
            f.write(line)
            f.write(fasta) 

In [32]:
makeFASTAfile(df_1, '../data/fasta.txt')

In [33]:
def blastLocal(fasta_path, out_path, evalue=10.0, window_size=3):
    start = datetime.datetime.now()  # for counting time necessary to run blast
    cmd1 = '../ncbi/blast/bin/'
    cmd2 = 'blastp' + ' '\
         + '-query ' + fasta_path + ' '\
         + '-db ' + cmd1 + 'pdbaa' + ' '\
         + '-evalue ' + str(evalue) + ' '\
         + '-outfmt ' + '5' + ' '\
         + '-out ' + out_path
    cmd = cmd1 + cmd2
    b_cmd = os.system(cmd)
    end = datetime.datetime.now()
    time = end - start
    c = divmod(time.days * 86400 + time.seconds, 60)
    print(cmd + ' : ran with exit code %d' %b_cmd)
    print(f'Running Blast took {c[0]} minutes {c[1]} seconds')

In [88]:
fasta_path = '../data/fasta.txt'  # path for the fasta file
out_path = '../data/blast_result.xml'  # path for the output file
evalue = 10.0
size = 3
blastLocal(fasta_path, out_path, evalue, size)

../ncbi/blast/bin/blastp -query ../data/fasta.txt -db ../ncbi/blast/bin/pdbaa -evalue 10.0 -outfmt 5 -out ../data/blast_result.xml : ran with exit code 0
(21, 58)


In [34]:
class BlastHandler(object):
    def __init__(self):
        self.dictlist = {'pdb_ID': [], 'BLAST_evalue': [], 'hit_from': [], 'hit_to': []}
        self.is_hit_id = False
        self.is_evalue = False
        self.is_hit_from = False
        self.is_hit_to = False
        self.ct_iter = 0
        self.ct = 0
        
    def start(self, tag, attrs):
        if tag == 'Hit':  # there are a few <Hsp> tags within a <Hit> tag
            self.ct_iter += 1
        elif tag == 'Hit_id':
            self.is_hit_id = True
        elif tag == 'Hsp_evalue':
            self.is_evalue = True
        elif tag == 'Hsp_hit-from':
            self.is_hit_from = True
        elif tag == 'Hsp_hit-to':
            self.is_hit_to = True
            
    def end(self, tag):
        if tag == 'Iteration':
            if self.ct_iter == 0:  # when there is no hit, add None to each list
                self.dictlist['pdb_ID'].append(None)
                self.dictlist['BLAST_evalue'].append(None)
                self.dictlist['hit_from'].append(None)
                self.dictlist['hit_to'].append(None)
            self.ct_iter = 0
            self.ct += 1
            if self.ct % 10000 == 0:
                print(f'coutner: {self.ct}')
        elif tag == 'Hit_id':
            self.is_hit_id = False
        elif tag == 'Hsp_evalue':
            self.is_evalue = False
        elif tag == 'Hsp_hit-from':
            self.is_hit_from = False
        elif tag == 'Hsp_hit-to':
            self.is_hit_to = False
        elif tag == 'Hsp':
            self.ct_iter += 1
        
    def data(self, data):
        if self.ct_iter == 1:
            if self.is_hit_id:
                try:
                    pdb = data.split('pdb|')[1]
                except: 
                    print(f'Cannot split pdb {data}')
                    pdb = None
                self.dictlist['pdb_ID'].append(pdb)
            elif self.is_evalue:
                self.dictlist['BLAST_evalue'].append(data)
            elif self.is_hit_from:
                self.dictlist['hit_from'].append(data)
            elif self.is_hit_to:
                self.dictlist['hit_to'].append(data)
    
    def close(self):
        print('Has completed parsing the blast_result xml')
        print(f'Total Count: {self.ct}')
        print(f'Length pdb: {len(self.dictlist["pdb_ID"])}, evalue: {len(self.dictlist["BLAST_evalue"])}, hit_from : {len(self.dictlist["hit_from"])}, hit_to : {len(self.dictlist["hit_to"])}')
        return self.dictlist

In [35]:
def readBlastXML(input_path):
    print('Start parcing')
    parser = etree.XMLParser(target=BlastHandler())
    data = etree.parse(input_path, parser)
    return data    

In [36]:
def combineBlastResults(df, dictlist):
    df['pdb_ID'] = dictlist['pdb_ID']
    df['BLAST_evalue'] = dictlist['BLAST_evalue']
    df['hit_from'] = dictlist['hit_from']
    df['hit_to'] = dictlist['hit_to']
    return df

In [96]:
blast_dl = readBlastXML('../data/blast_result.xml')
blast_df = pd.DataFrame(blast_dl)
print(len(blast_dl['pdb_ID']))
blast_df.head()

Start parcing
coutner: 10000
Length pdb: 10000, evalue: 10000, hit_from : 10000, hit_to : 10000
coutner: 20000
Length pdb: 20000, evalue: 20000, hit_from : 20000, hit_to : 20000
coutner: 30000
Length pdb: 30000, evalue: 30000, hit_from : 30000, hit_to : 30000
coutner: 40000
Length pdb: 40000, evalue: 40000, hit_from : 40000, hit_to : 40000
coutner: 50000
Length pdb: 50000, evalue: 50000, hit_from : 50000, hit_to : 50000
coutner: 60000
Length pdb: 60000, evalue: 60000, hit_from : 60000, hit_to : 60000
coutner: 70000
Length pdb: 70000, evalue: 70000, hit_from : 70000, hit_to : 70000
Has completed parsing the blast_result xml
Total Count: 70575
Length pdb: 70575, evalue: 70575, hit_from : 70575, hit_to : 70575
70575


,pdb_ID,BLAST_evalue,hit_from,hit_to
0,1APY|A,6.44088e-11,25,49
1,1GT8|A,1.87182e-09,874,898
2,3I2B|A,1.89206e-10,3,24
3,4DT7|E,3.1355e-07,1,19
4,1FSU|A,3.56958e-11,84,108


In [37]:
df_1 = pd.read_csv('../data/MM_enzyme.csv')
df_2 = combineBlastResults(df_1, blast_dl)
df_2.to_csv('../data/MM_enzyme_blast.csv', index = False, header = True)
df_2.head()

NameError: name 'blast_dl' is not defined

In [38]:
# when you don't run but just import csv file
df_2 = pd.read_csv('../data/MM_enzyme_blast.csv')
df_2.head()

,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,FASTA_window,pdb_ID,BLAST_evalue,hit_from,hit_to
0,AGA,aspartylglucosaminidase,uncertain significance,3.5.1.26,G60D,NP_000018.2,VCV000000222,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS,1APY|A,6.440880e-11,25.0,49.0
1,DPYD,dihydropyrimidine dehydrogenase,uncertain significance,1.3.1.2,R886H,NP_000101.2,VCV000000437,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK,1GT8|A,1.871820e-09,874.0,898.0
2,PTS,6-pyruvoyltetrahydropterin synthase,uncertain significance,4.2.3.12,R16C,NP_000308.1,VCV000000477,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS,3I2B|A,1.892060e-10,3.0,24.0
3,PROC,"protein C, inactivator of coagulation factors ...",uncertain significance,3.4.21.69,P210L,NP_000303.1,VCV000000661,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD,4DT7|E,3.135500e-07,1.0,19.0
4,ARSB,arylsulfatase B,uncertain significance,3.1.6.12,G137V,NP_000037.2,VCV000000877,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG,1FSU|A,3.569580e-11,84.0,108.0


In [129]:
def orderColumns(df, col_names):
    df_cols = set(df.columns)
    if set(col_names).issubset(df_cols):
            df_ordered = df.loc[:, col_names]
            return df_ordered
    else:
        print('column names given include missing labels')

In [132]:
columns = ('gene_ID', 'gene_name', 'clinical_significance', 'EC_number', 'missense_variation', 'NP_accession', 'ClinVar_accession', 'gnomAD_AF', 'SIFT_prediction', 'CADD_score', 'Chr', 'start', 'stop', 'referenceAllele', 'alternateAllele', 'FASTA_window', 'pdb_ID', 'BLAST_evalue', 'hit_from', 'hit_to')
columns2 = ('gene_ID', 'gene_name', 'clinical_significance', 'EC_number', 'missense_variation', 'NP_accession', 'ClinVar_accession', 'Chr', 'start', 'stop', 'referenceAllele', 'alternateAllele', 'FASTA_window', 'pdb_ID', 'BLAST_evalue', 'hit_from', 'hit_to')
df_3 = orderColumns(df_2, columns2)
df_3.head()

,gene_ID,gene_name,clinical_significance,EC_number,missense_variation,NP_accession,ClinVar_accession,Chr,start,stop,referenceAllele,alternateAllele,FASTA_window,pdb_ID,BLAST_evalue,hit_from,hit_to
0,AGA,aspartylglucosaminidase,uncertain significance,3.5.1.26,G60D,NP_000018.2,VCV000000222,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS,1APY|A,6.44088e-11,25,49
1,DPYD,dihydropyrimidine dehydrogenase,uncertain significance,1.3.1.2,R886H,NP_000101.2,VCV000000437,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK,1GT8|A,1.87182e-09,874,898
2,PTS,6-pyruvoyltetrahydropterin synthase,uncertain significance,4.2.3.12,R16C,NP_000308.1,VCV000000477,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS,3I2B|A,1.89206e-10,3,24
3,PROC,"protein C, inactivator of coagulation factors ...",uncertain significance,3.4.21.69,P210L,NP_000303.1,VCV000000661,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD,4DT7|E,3.1355e-07,1,19
4,ARSB,arylsulfatase B,uncertain significance,3.1.6.12,G137V,NP_000037.2,VCV000000877,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG,1FSU|A,3.56958e-11,84,108


In [39]:
def makeInputForVep(df, output_path):
    df_sub = df[['Chr', 'start', 'stop', 'referenceAllele', 'alternateAllele']]
    df_sub.to_csv(output_path, sep='\t', index=False, header=False)
    return df_sub

In [40]:
vep_input_path = '../data/vep_input.txt'
df_2_sub = makeInputForVep(df_2, vep_input_path)
df_2_sub.head()

,Chr,start,stop,referenceAllele,alternateAllele
0,4,177440375,177440375,C,T
1,1,97098598,97098598,C,T
2,11,112226489,112226489,C,T
3,2,127426178,127426178,C,T
4,5,78969095,78969095,C,A


In [61]:
def vepLocal(input_path, output_path):
    start = datetime.datetime.now()  # for counting time necessary to run blast
    cmd1 = '../../ensembl-vep/'  # specify directory
    cmd2 = './vep' + ' '\
         + '-i ' + input_path + ' '\
         + '-o ' + output_path + ' '\
         + '--cache ' + '--af_gnomad '\
         + '--appris ' + '--canonical '\
         + '--sift p ' + '--polyphen p '\
         + '--no_check_variants_order '\
         + '--flag_pick ' + '--tab '\
         + '--force_overwrite'
    cmd = cmd1 + cmd2
    vep_cmd = os.system(cmd)
    end = datetime.datetime.now()
    time = end - start
    c = divmod(time.days * 86400 + time.seconds, 60)
    print(cmd + ' : ran with exit code %d' %vep_cmd)
    print(f'Running Vep took {c[0]} minutes {c[1]} seconds')

In [58]:
vep_output_path = '../data/vep_output.txt'
vepLocal(vep_input_path, vep_output_path)

True
../../ensembl-vep/./vep -i ../data/vep_input.txt -o ../data/vep_output.txt --cache --af_gnomad --appris --canonical --sift p --polyphen p --flag_pick --tab --force_overwrite : ran with exit code 2
Running Vep took 0 minutes 8 seconds


In [66]:
# vep demo
vep_output_path_demo = '../data/vep_output_short.txt'
vepLocal('../data/inputForVep_short.txt', vep_output_path_demo)

../../ensembl-vep/./vep -i ../data/inputForVep_short.txt -o ../data/vep_output_short.txt --cache --af_gnomad --appris --canonical --sift p --polyphen p --no_check_variants_order --flag_pick --tab --force_overwrite : ran with exit code 0
Running Vep took 0 minutes 41 seconds


In [70]:
def cropVepOutput(input_path, output_path):
    cmd = "cat " + input_path + " | "\
        + "grep -v '##'" + " | "\
        + "sed 's/^#\(.*\)/\\1/'" +  " >| "\
        + output_path
    crop_cmd = os.system(cmd)
    print(cmd + ' : ran with exit code %d' %crop_cmd)

In [ ]:
crop_vep_path = '../data/vep_output_cropped.txt'
cropVepOutput(vep_output_path, crop_vep_path)

In [71]:
# vep demo
crop_vep_path_demo = '../data/vep_output_short_cropped.txt'
cropVepOutput(vep_output_path_demo, crop_vep_path_demo)

cat ../data/vep_output_short.txt | grep -v '##' | sed 's/^#\(.*\)/\1/' >| ../data/vep_output_short_cropped.txt : ran with exit code 0


In [76]:
df_vep = pd.read_csv(crop_vep_path_demo, sep='\t', header=0)
df_vep.head()

,Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,...,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,CLIN_SIG,SOMATIC,PHENO
0,155238261,1:155238261,T,ENSG00000177628,ENST00000327247,Transcript,missense_variant,867,634,212,...,-,-,-,-,-,-,-,uncertain_significance,-,1
1,155238261,1:155238261,T,ENSG00000177628,ENST00000368373,Transcript,missense_variant,771,634,212,...,-,-,-,-,-,-,-,uncertain_significance,-,1
2,155238261,1:155238261,T,ENSG00000177628,ENST00000427500,Transcript,missense_variant,650,487,163,...,-,-,-,-,-,-,-,uncertain_significance,-,1
3,155238261,1:155238261,T,ENSG00000177628,ENST00000428024,Transcript,missense_variant,713,373,125,...,-,-,-,-,-,-,-,uncertain_significance,-,1
4,155238261,1:155238261,T,ENSG00000236675,ENST00000440904,Transcript,downstream_gene_variant,-,-,-,...,-,-,-,-,-,-,-,uncertain_significance,-,1


In [92]:
df_vep_pick = df_vep.loc[df_vep['PICK'] == '1']
df_vep_pick.head()

,Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,...,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,CLIN_SIG,SOMATIC,PHENO
0,155238261,1:155238261,T,ENSG00000177628,ENST00000327247,Transcript,missense_variant,867,634,212,...,-,-,-,-,-,-,-,uncertain_significance,-,1
16,100206558,1:100206558,G,ENSG00000137992,ENST00000370132,Transcript,missense_variant,1110,1096,366,...,-,-,-,-,-,-,-,uncertain_significance,-,1
17,100196274,1:100196274,C,ENSG00000137992,ENST00000370132,Transcript,missense_variant,1444,1430,477,...,0,0,0,0,8.809e-06,0,0,"uncertain_significance,pathogenic",-,"1,1"
20,226888945,1:226888945,T,ENSG00000143801,ENST00000366783,Transcript,missense_variant,1066,683,228,...,0,0,0,0,1.759e-05,0,0,not_provided,-,"1,1"
28,94080562,1:94080562,C,ENSG00000198691,ENST00000370225,Transcript,missense_variant,1118,1015,339,...,0,0,0,0,1.763e-05,0,0,conflicting_interpretations_of_pathogenicity,-,"1,1"


In [ ]:
# ./vep -i inputForVep_short.txt -o outputForVep_short.txt --cache --af_gnomad --appris --canonical --sift p --polyphen p --force_overwrite --tab --flag_pick --no_check_variants_order
# ./vep -i inputForVep.txt -o outputForVep.txt --cache --af_gnomad --appris --canonical --sift p --polyphen p --force_overwrite --tab --flag_pick

In [28]:
# https://cadd.gs.washington.edu/api/v1.0/GRCh38-v1.4/22:44044001_T_A

In [29]:
# #Specify score file link
# SCORE_FILE=http://krishna.gs.washington.edu/download/CADD/v1.4/GRCh37/whole_genome_SNVs_inclAnno.tsv.gz
# #Download Tabix Index
# INDEX=IndexFile
# wget -c $SCORE_FILE.tbi -O $INDEX

In [133]:
!jupyter nbconvert --to script VersUS.ipynb
with open('VersUS.py', 'r') as f:
    lines = f.readlines()
with open('VersUS.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to script' in line:
            break
        else:
            f.write(line)

[NbConvertApp] Converting notebook VersUS.ipynb to script
[NbConvertApp] Writing 21911 bytes to VersUS.py
